### Library

In [1]:
import sys
import os
from pathlib import Path

# 프로젝트 루트 경로 추가
project_root = Path().absolute().parent
os.chdir(project_root)
sys.path.insert(0, str(project_root))

from omegaconf import OmegaConf
from src.main import LegalQAEngine

### Configuration Setup

In [2]:
# Hydra를 사용한 설정 파일 로드
import hydra
from hydra import compose, initialize_config_dir
from omegaconf import OmegaConf

# 현재 작업 디렉토리 확인
print(f"Current working directory: {os.getcwd()}")

# Hydra 초기화 및 설정 로드
config_dir = project_root / "conf"
print(f"Config directory: {config_dir}")

# 기존 Hydra 인스턴스가 있다면 정리
if hydra.core.global_hydra.GlobalHydra.instance().is_initialized():
    hydra.core.global_hydra.GlobalHydra.instance().clear()

# Hydra로 설정 구성
with initialize_config_dir(config_dir=str(config_dir), version_base=None):
    cfg = compose(config_name="config")

print("Hydra 설정 로드 완료")
print(OmegaConf.to_yaml(cfg))

Current working directory: /Users/mungughyeon/Documents/Upstage/Contest/4. Langchain/upstageailab-langchain-pjt-langchain_1
Config directory: /Users/mungughyeon/Documents/Upstage/Contest/4. Langchain/upstageailab-langchain-pjt-langchain_1/conf
Hydra 설정 로드 완료
llm:
  provider: google
  model_name: gemini-1.5-flash
  temperature: 0.7
embedding:
  provider: google
  model_name: models/embedding-001
data:
  path: data/cases_2021_2023.csv
  chunk_size: 2000
  chunk_overlap: 200
langsmith:
  enabled: true
  project_name: legal-qa-chatbot
  endpoint: https://api.smith.langchain.com
vector_store:
  type: faiss
  persist_directory: ./faiss_db
retriever:
  type: ensemble
  weights:
  - 0.5
  - 0.5
chain:
  type: retrieval_qa
  retriever_k: 4
preprocessing:
  enabled: true
  filter_case_types: false
  target_case_types: []
chat_history:
  max_history: 5
  save_path: logs/chat_history.json
logging:
  level: INFO
  directory: logs
interactive: false



### QA Engine Initialization

In [3]:
# QA 엔진 초기화
qa_engine = LegalQAEngine(cfg)
print("QA 엔진 초기화 완료")

2025-08-25 18:20:31,850 - LegalQA_Main - INFO - __init__:57 - Logger 'LegalQA_Main' initialized with level INFO
2025-08-25 18:20:31,851 - LegalQA_LLM - INFO - __init__:57 - Logger 'LegalQA_LLM' initialized with level INFO
2025-08-25 18:20:31,851 - LegalQA_VectorStore - INFO - __init__:57 - Logger 'LegalQA_VectorStore' initialized with level INFO
2025-08-25 18:20:31,852 - LegalQA_ChatHistory - INFO - __init__:57 - Logger 'LegalQA_ChatHistory' initialized with level INFO
2025-08-25 18:20:31,853 - LegalQA_Retriever - INFO - __init__:57 - Logger 'LegalQA_Retriever' initialized with level INFO
2025-08-25 18:20:31,853 - LegalQA_Main - INFO - _setup_environment:57 - LangSmith 추적 활성화: legal-qa-chatbot
2025-08-25 18:20:31,854 - LegalQA_Main - INFO - log_system_event:82 - System Event - llm_initialization_start: None
2025-08-25 18:20:31,869 - LegalQA_Main - INFO - log_system_event:82 - System Event - document_loading_start: None
전처리 전 문서 수: 51
기본 전처리 후 문서 수: 51
전처리 완료
2025-08-25 18:20:31,916 - L

### System Status Check

In [4]:
# 시스템 상태 확인
status = qa_engine.get_system_status()

for key, value in status.items():
    print(f"{key}: {value}")

I0000 00:00:1756113653.636224  761875 fork_posix.cc:71] Other threads are currently calling into gRPC, skipping fork() handlers


initialized: True
timestamp: 2025-08-25 18:20:53
llm: {'status': 'success', 'provider': 'google', 'model': 'gemini-1.5-flash', 'response_valid': True, 'test_response': "content='Hello there! How can I help you today?' additional_kwargs={} response_metadata={'prompt_fee"}
vectorstore: {'type': 'faiss', 'status': 'ready', 'total_vectors': 194}
chat_history: {'session_id': '20250825_182046', 'total_messages': 0, 'user_messages': 0, 'assistant_messages': 0, 'qa_pairs': 0, 'max_history': 5, 'memory_usage': '0/10'}
logging: {'total_logs': 4, 'log_types': {'system_event': 4}, 'session_duration': 'active'}


### Basic QA Test

In [5]:
# 기본 질문 테스트
test_question = "사업주가 근로기준법을 위반했을 때 어떤 처벌을 받게 돼?"
result = qa_engine.ask(test_question)

print(f"질문: {result['question']}")
print(f"답변: {result['answer']}")
print(f"응답 시간: {result['response_time']:.2f}초")
print(f"성공 여부: {result['success']}")

2025-08-25 18:22:57,487 - LegalQA_Main - INFO - log_qa_interaction:70 - QA Interaction - Q: 사업주가 근로기준법을 위반했을 때 어떤 처벌을 받게 돼?... | A: ## 관련 법령 및 조항
- **근로기준법**: 근로기준법은 근로자의 근로조건, 임금, 휴...
질문: 사업주가 근로기준법을 위반했을 때 어떤 처벌을 받게 돼?
답변: ## 관련 법령 및 조항
- **근로기준법**: 근로기준법은 근로자의 근로조건, 임금, 휴일, 해고 등에 관한 기본적인 사항을 규정하고 있습니다.  사업주가 이 법을 위반하면 다양한 처벌을 받게 됩니다.  구체적인 위반 행위와 처벌 내용은 위반 내용에 따라 다릅니다.  예를 들어, 임금 체불은 형사처벌과 민사상 손해배상 책임을 동시에 질 수 있습니다.  또한, 부당해고는 부당해고구제 절차를 통해 원직복직이나 손해배상을 명령받을 수 있습니다.

- **근로기준법 제109조**: 사업주가 근로기준법을 위반한 경우, 벌금 또는 과태료 처분을 받을 수 있습니다.  구체적인 금액은 위반 행위의 종류와 정도에 따라 다릅니다.


## 유사 판례 분석
제공된 판례 데이터에는 근로기준법 위반에 대한 사업주 처벌에 직접적으로 언급하는 판례가 없습니다.  제공된 판례들은 변호사법 위반, 퇴직금, 부당해고, 임금 등 개별적인 근로 문제에 대한 판결입니다.  따라서 근로기준법 위반에 따른 사업주 처벌에 대한 구체적인 판례 분석은 불가능합니다.


## 법적 조언
사업주가 근로기준법을 위반했을 때 받는 처벌은 위반 내용에 따라 다릅니다.  임금체불,  부당해고,  근로시간 위반 등 각 위반 행위에 따른 처벌 규정을 근로기준법 및 관련 법령에서 확인해야 합니다.  

사업주가 근로기준법을 위반한 경우, 근로자는 다음과 같은 방법으로 권리 구제를 받을 수 있습니다.

1. **민사소송**: 임금체불,  손해배상 청구 등 민사상 손해배상을 청구할 수 있습니다.
2. **행정절차**: 고용노동

### Chat History Test

In [6]:
# 연속 질문으로 히스토리 테스트
questions = [
    "임대차계약에서 보증금은 언제 돌려받을 수 있나요?",
    "그럼 임대인이 보증금을 안 돌려주면 어떻게 해야 하나요?",
    "앞서 말한 절차에서 필요한 서류는 무엇인가요?"
]

for question in questions:
    print(f"\n--- 새 질문 ---")
    result = qa_engine.ask(question, use_chat_history=True)
    print(f"Q: {question}")
    print(f"A: {result['answer'][:200]}...")
    print(f"응답시간: {result['response_time']:.2f}초")


--- 새 질문 ---
2025-08-25 18:23:18,630 - LegalQA_Main - INFO - log_qa_interaction:70 - QA Interaction - Q: 임대차계약에서 보증금은 언제 돌려받을 수 있나요?... | A: ## 관련 법령 및 조항

- **주택임대차보호법 제3조(임대차의 목적)**: 임대차의 목...
Q: 임대차계약에서 보증금은 언제 돌려받을 수 있나요?
A: ## 관련 법령 및 조항

- **주택임대차보호법 제3조(임대차의 목적)**: 임대차의 목적은 주택입니다.  본 조항은 임대차 계약의 기본적인 대상을 규정합니다.

- **주택임대차보호법 제11조(차임 및 보증금의 지급)**: 임대인은 임차인에게 주택을 인도하고 임차인은 임대인에게 차임 및 보증금을 지급해야 합니다.  보증금 반환 시점은 계약 종료 시점입니...
응답시간: 6.11초

--- 새 질문 ---
2025-08-25 18:23:25,232 - LegalQA_Main - INFO - log_qa_interaction:70 - QA Interaction - Q: 그럼 임대인이 보증금을 안 돌려주면 어떻게 해야 하나요?... | A: ## 관련 법령 및 조항

- **주택임대차보호법 제3조(임대차의 목적)**: 임대차의 목...
Q: 그럼 임대인이 보증금을 안 돌려주면 어떻게 해야 하나요?
A: ## 관련 법령 및 조항

- **주택임대차보호법 제3조(임대차의 목적)**: 임대차의 목적은 주택입니다.  본 조항은 임대차 계약의 기본적인 대상을 규정합니다.

- **주택임대차보호법 제11조(차임 및 보증금의 지급)**: 임대인은 임차인에게 주택을 인도하고 임차인은 임대인에게 차임 및 보증금을 지급해야 합니다.  보증금 반환 시점은 계약 종료 시점입니...
응답시간: 6.60초

--- 새 질문 ---
2025-08-25 18:23:33,912 - LegalQA_Main - INFO - log_qa_interaction:70 - QA Interaction

### Chat History Status

In [7]:
# 채팅 히스토리 상태 확인
history_stats = qa_engine.chat_history.get_stats()

print("채팅 히스토리 통계:")
for key, value in history_stats.items():
    print(f"{key}: {value}")

채팅 히스토리 통계:
session_id: 20250825_182046
total_messages: 8
user_messages: 4
assistant_messages: 4
qa_pairs: 4
max_history: 5
memory_usage: 8/10


### Recent QA Pairs

In [8]:
# 최근 질답 쌍 확인
recent_pairs = qa_engine.chat_history.get_recent_qa_pairs(num_pairs=2)

print("최근 질답 쌍:")
for pair in recent_pairs:
    print(f"\n--- 질답 쌍 ---")
    print(f"Q: {pair['question']}")
    print(f"A: {pair['answer'][:100]}...")
    print(f"시간: {pair['timestamp']}")

최근 질답 쌍:

--- 질답 쌍 ---
Q: 그럼 임대인이 보증금을 안 돌려주면 어떻게 해야 하나요?
A: ## 관련 법령 및 조항

- **주택임대차보호법 제3조(임대차의 목적)**: 임대차의 목적은 주택입니다.  본 조항은 임대차 계약의 기본적인 대상을 규정합니다.

- **주택임대...
시간: 2025-08-25T18:23:25.231355

--- 질답 쌍 ---
Q: 앞서 말한 절차에서 필요한 서류는 무엇인가요?
A: ## 관련 법령 및 조항

- **주택임대차보호법 제3조(임대차의 목적)**: 임대차 계약의 기본적인 대상을 규정합니다.
- **주택임대차보호법 제11조(차임 및 보증금의 지급)*...
시간: 2025-08-25T18:23:33.906159


### Vector Store Information

In [9]:
# 벡터 스토어 정보 확인
vectorstore_stats = qa_engine.vectorstore_manager.get_stats()

print("벡터 스토어 정보:")
for key, value in vectorstore_stats.items():
    print(f"{key}: {value}")

벡터 스토어 정보:
type: faiss
status: ready
total_vectors: 194


### Logging Statistics

In [10]:
# 로깅 통계 확인
log_stats = qa_engine.logger.get_session_stats()

print("로깅 통계:")
for key, value in log_stats.items():
    print(f"{key}: {value}")

로깅 통계:
total_logs: 8
log_types: {'system_event': 4, 'qa_interaction': 4}
session_duration: active


### Performance Test

In [11]:
# 성능 테스트
import time

performance_questions = [
    "교통사고 손해배상 기준은?",
    "근로계약 해지 절차는?",
    "부동산 매매계약 주의사항은?"
]

total_time = 0
results = []

for question in performance_questions:
    result = qa_engine.ask(question, use_chat_history=False)
    total_time += result['response_time']
    results.append({
        'question': question,
        'time': result['response_time'],
        'success': result['success']
    })

print(f"총 질문 수: {len(performance_questions)}")
print(f"총 소요 시간: {total_time:.2f}초")
print(f"평균 응답 시간: {total_time/len(performance_questions):.2f}초")
print(f"성공률: {sum(1 for r in results if r['success'])/len(results):.2%}")

print("\n개별 결과:")
for result in results:
    print(f"  {result['question']}: {result['time']:.2f}초 ({'성공' if result['success'] else '실패'})")

2025-08-25 18:24:19,400 - LegalQA_Main - INFO - log_qa_interaction:70 - QA Interaction - Q: 교통사고 손해배상 기준은?... | A: ## 관련 법령 및 조항

- **민법 제750조 (불법행위로 인한 손해배상):**  불법...
2025-08-25 18:24:27,271 - LegalQA_Main - INFO - log_qa_interaction:70 - QA Interaction - Q: 근로계약 해지 절차는?... | A: ## 관련 법령 및 조항
- **근로기준법 제23조(해고의 제한):** 사용자는 근로자를 ...
2025-08-25 18:24:35,532 - LegalQA_Main - INFO - log_qa_interaction:70 - QA Interaction - Q: 부동산 매매계약 주의사항은?... | A: ## 관련 법령 및 조항
- **민법 제535조(매매의 의의):** 매매는 당사자 일방이 ...
총 질문 수: 3
총 소요 시간: 22.05초
평균 응답 시간: 7.35초
성공률: 100.00%

개별 결과:
  교통사고 손해배상 기준은?: 5.93초 (성공)
  근로계약 해지 절차는?: 7.87초 (성공)
  부동산 매매계약 주의사항은?: 8.26초 (성공)


### Final System Summary

In [12]:
# 최종 시스템 요약
final_status = qa_engine.get_system_status()

print("=== 법률 QA 시스템 최종 상태 ===")
print(f"초기화 상태: {final_status['initialized']}")
print(f"타임스탬프: {final_status['timestamp']}")

if 'llm' in final_status:
    print(f"LLM 상태: {final_status['llm']['status']}")
    print(f"모델: {final_status['llm']['provider']}/{final_status['llm']['model']}")

if 'vectorstore' in final_status:
    print(f"벡터스토어: {final_status['vectorstore']['status']}")

if 'chat_history' in final_status:
    history = final_status['chat_history']
    print(f"채팅 히스토리: {history['qa_pairs']}개 질답쌍 저장됨")

print("\n테스트 완료!")

=== 법률 QA 시스템 최종 상태 ===
초기화 상태: True
타임스탬프: 2025-08-25 18:24:50
LLM 상태: success
모델: google/gemini-1.5-flash
벡터스토어: ready
채팅 히스토리: 4개 질답쌍 저장됨

테스트 완료!
